In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from tpot import TPOTClassifier
from sklearn.model_selection import cross_val_score

In [23]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
pd.set_option('display.max_rows', 10)
train.set_index(['user_id'], inplace=True)
test.set_index(['user_id'], inplace=True)

In [24]:
lenc = LabelEncoder()
for col in train.columns:
    if train[col].dtype == 'object':
        l = lenc.fit_transform(train[col])
        train[col] = l
for col in test.columns:
    if test[col].dtype == 'object':
        l = lenc.fit_transform(test[col])
        test[col] = l

In [25]:
train_x = train.drop(['Attrition', 'EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis=1)
train_y = train['Attrition']

In [27]:
# Logistic Regression
lr = LogisticRegression(solver='liblinear', multi_class='auto')
lr.fit(train_x, train_y)
predict_y = lr.predict(test.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis=1))

test_y = test.drop(test.columns[0:34], axis=1, inplace=False)
test_y.insert(0, 'Attrition', predict_y)
test_y = test_y.reset_index()
test_y.to_csv('./lr.csv', index=False)

In [32]:
# 得到准确率(基于训练集)
acc = round(lr.score(train_x, train_y), 6)
print(u'score准确率为 %.4lf' % acc)
# 使用K折交叉验证 统计准确率
print(u'cross_val_score准确率为 %.4lf' % np.mean(cross_val_score(lr, train_x, train_y, cv=10)))

score准确率为 0.8682
cross_val_score准确率为 0.8571


In [20]:
# Decision Tree
'''clf = DecisionTreeClassifier(criterion='gini')
clf.fit(train_x, train_y)
predict_y = clf.predict(test)

test_y = test.drop(test.columns[0:34], axis=1, inplace=False)
test_y.insert(0, 'Attrition', predict_y)
test_y = test_y.reset_index()
test_y.to_csv('./clf.csv', index=False)'''

In [13]:
# TPOT
tpot = TPOTClassifier(generations=20, population_size=30, verbosity=2)
tpot.fit(train_x, train_y)
tpot.export('tpot_attrition_pipeline.py')

Generation 1 - Current best internal CV score: 0.8707356653443924
Generation 2 - Current best internal CV score: 0.8707356653443924
Generation 3 - Current best internal CV score: 0.873306887847097
Generation 4 - Current best internal CV score: 0.8749765596826542
Generation 5 - Current best internal CV score: 0.8749765596826542
Generation 6 - Current best internal CV score: 0.8749765596826542
Generation 7 - Current best internal CV score: 0.8749765596826542
Generation 8 - Current best internal CV score: 0.8750018031013342
Generation 9 - Current best internal CV score: 0.8758276235124413
Generation 10 - Current best internal CV score: 0.8758276235124413
Generation 11 - Current best internal CV score: 0.8775369635773531
Generation 12 - Current best internal CV score: 0.8775369635773531
Generation 13 - Current best internal CV score: 0.8775369635773531
Generation 14 - Current best internal CV score: 0.8817886765236207
Generation 15 - Current best internal CV score: 0.8817886765236207
Gener